In [22]:
from get_data import get_data
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

import time

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\images.pt'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

data, references, images, descriptions, target = get_data(data_month, desc_path, img_path)

modello = sp.create_model(len(data.columns)).to(device)
batch_size = 128
epochs = 100
criterion = nn.MSELoss()
early_stop = sp.EarlyStopping(5, 0.001)

optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-3},
        {"params": modello.tabular.parameters()},
        {"params": modello.description.parameters(), "lr": 1e-3},
        {"params": modello.final.parameters()}
    ],
    lr=1e-2,
)

train, val = getDataset(references, data, images, descriptions, target, batch_size, 1)

folder = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\month_negozi_prova'
if not os.path.isdir(folder):
    os.makedirs(folder)

mse_final, mae_final, r2_final = 0, 0, 0
min_mae = 100000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = sp.validation_loop(val, modello, criterion, device)
    if (min_mae > mae):
        model_sc = torch.jit.script(modello)
        time.sleep(1)
        model_sc.save(f'{folder}\\weights.pt')
        min_mae = mae
        print(f'MINIMA MAE AGGIORNATA: {min_mae}')
        
    early_stp = early_stop(mse)
    
    if early_stp:
        print('Early Stop attivato.')
        break
    
print("Fatto!")

Sto usando cuda
Epoch 1
-------------------------------
MSE: 14955.312500  [  128/27550]


c:\Users\GRVRLD00P\Documents\Progetto ORS\Progetto ORS\ProgettoORS\sales_prediction\get_data.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  descrizioni = torch.load(des

MSE: 5539.208984  [12928/27550]
MSE: 5458.679688  [25728/27550]
In questa epoca, l'MSE medio è 7148.603515625
Validation Error: 
 Avg MSE: 5941.114980 
 Avg MAE: 57.429989 
 R2: 0.118277
 Bias: 11.266496

MINIMA MAE AGGIORNATA: 57.429989435339486
Epoch 2
-------------------------------
MSE: 7245.271484  [  128/27550]
MSE: 4324.251953  [12928/27550]
MSE: 5051.924805  [25728/27550]
In questa epoca, l'MSE medio è 5190.44580078125
Validation Error: 
 Avg MSE: 5467.336373 
 Avg MAE: 53.217605 
 R2: 0.189372
 Bias: 8.124609

MINIMA MAE AGGIORNATA: 53.21760468841881
Epoch 3
-------------------------------
MSE: 6629.404297  [  128/27550]
MSE: 4200.442871  [12928/27550]
MSE: 4360.661621  [25728/27550]
In questa epoca, l'MSE medio è 4891.10205078125
Validation Error: 
 Avg MSE: 4784.710680 
 Avg MAE: 48.376954 
 R2: 0.290174
 Bias: 3.812769

MINIMA MAE AGGIORNATA: 48.376953781292
Epoch 4
-------------------------------
MSE: 6503.898438  [  128/27550]
MSE: 3392.322754  [12928/27550]
MSE: 4108.882